In [274]:
import pandas as pd
import numpy as np

In [275]:
df = pd.read_csv(r'..\..\datasets\1. Originales\TLC Aggregated Data\data_reports_monthly.csv')

In [276]:
df.columns = [
    'date', 'industry', 'trips_per_day', 'farebox_per_day',
    'unique_drivers', 'unique_vehicles', 'vehicles_per_day',
    'avg_days_vehicles_on_road', 'avg_hours_per_day_per_vehicle',
    'avg_days_drivers_on_road', 'avg_hours_per_day_per_driver',
    'avg_minutes_per_trip', 'percent_of_trips_paid_with_credit_card',
    'trips_per_day_shared'
]

In [277]:
# Columnas incuidas
columns_to_replace = df.columns.difference(['date', 'industry'])
# Reemplazar '-' por '' en las columnas seleccionadas
df[columns_to_replace] = df[columns_to_replace].replace({'-':np.nan,',':'','%': ''}, regex=True)

In [278]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m')
df['industry'] = df['industry'].astype('category')
df['trips_per_day'] = df['trips_per_day'].astype('Int64')
df['farebox_per_day'] = df['farebox_per_day'].astype('Int64')
df['unique_drivers'] = df['unique_drivers'].astype('Int64')
df['unique_vehicles'] = df['unique_vehicles'].astype('Int64')
df['vehicles_per_day'] = df['vehicles_per_day'].astype('Int64')
df['percent_of_trips_paid_with_credit_card'] = pd.to_numeric(df['percent_of_trips_paid_with_credit_card'], errors='coerce') / 100
df['trips_per_day_shared'] = pd.to_numeric(df['trips_per_day_shared'], errors='coerce') 

In [279]:
# Reemplazar 'Green' y 'Yellow' para igual al datset diario
df['industry'] = df['industry'].replace({'Green': 'Green Taxi', 'Yellow': 'Yellow Taxi'})

C:\Users\Hernán\AppData\Local\Temp\ipykernel_6528\1759112715.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df['industry'] = df['industry'].replace({'Green': 'Green Taxi', 'Yellow': 'Yellow Taxi'})


In [280]:
# Agregar "FHV - Other" a las categorías de la columna 'industry' para reemplazar las que no están en el datset diario  
df['industry'] = df['industry'].cat.add_categories('FHV - Other')

# Luego, realizar la asignación de las industrias
other_industries = ['FHV - Black Car', 'FHV - Livery', 'FHV - Lux Limo']
df.loc[df['industry'].isin(other_industries), 'industry'] = 'FHV - Other'

In [281]:
# Agrupar por 'month_year' y 'industry', y agregar 'FHV - Other' por mes
df_other = df[df['industry'] == 'FHV - Other'].groupby('date').agg({
    'trips_per_day': 'sum',
    'farebox_per_day': 'sum',
    'unique_drivers': 'sum',
    'unique_vehicles': 'sum',
    'vehicles_per_day': 'sum',
    'avg_days_vehicles_on_road': 'mean',
    'avg_hours_per_day_per_vehicle': 'mean',
    'avg_days_drivers_on_road': 'mean',
    'avg_hours_per_day_per_driver': 'mean',
    'avg_minutes_per_trip': 'mean',
    'percent_of_trips_paid_with_credit_card': 'mean',
    'trips_per_day_shared': 'sum'
}).reset_index()

# Reasignar índices y juntar los DataFrames
df_other['industry'] = 'FHV - Other'  # Asegurarse de que la columna 'industry' sea consistente
df = pd.concat([df[df['industry'] != 'FHV - Other'], df_other], ignore_index=True)

In [282]:
# df['year'] = df['date'].dt.year
# df['month'] = df['date'].dt.month

In [283]:
df[df['date'] == '2024-08-01']

,date,industry,trips_per_day,farebox_per_day,unique_drivers,unique_vehicles,vehicles_per_day,avg_days_vehicles_on_road,avg_hours_per_day_per_vehicle,avg_days_drivers_on_road,avg_hours_per_day_per_driver,avg_minutes_per_trip,percent_of_trips_paid_with_credit_card,trips_per_day_shared
0,2024-08-01,FHV - High Volume,617043,<NA>,81306,80570,55030,21.200000,6.300000,21.3,6.300000,19.000000,NaN,10185.0
1,2024-08-01,Green Taxi,1654,38632,728,694,394,17.600000,3.500000,17.0,3.400000,14.900000,0.72,NaN
2,2024-08-01,Yellow Taxi,94732,2360656,12276,9125,7822,26.600000,7.600000,22.6,6.600000,16.200000,0.81,NaN
540,2024-08-01,FHV - Other,47825,0,19842,19783,8831,12.966667,3.533333,13.0,3.466667,38.666667,NaN,0.0


In [284]:
df.to_csv(r'..\..\datasets\2. Depurados\TLC Aggregated Data\data_reports_monthly.csv',index=False)  

---
---
Junto los dos datasets en un solo archivo

In [285]:
df1 = pd.read_csv(r'..\..\datasets\2. Depurados\TLC Aggregated Data\viajes_depurado_by_industry.csv')
df2 = pd.read_csv(r'..\..\datasets\2. Depurados\TLC Aggregated Data\data_reports_monthly.csv')

In [286]:
df1['passenger_count'] = df1['passenger_count'].replace(0, np.nan)
df1['fare_amount'] = df1['fare_amount'].replace(0, np.nan)
df1['total_amount'] = df1['total_amount'].replace(0, np.nan)
df1['shared_requests'] = df1['shared_requests'].replace(0, np.nan)

In [287]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               176 non-null    int64  
 1   month              176 non-null    int64  
 2   industry           176 non-null    object 
 3   total_trips        176 non-null    int64  
 4   passenger_count    88 non-null     float64
 5   trip_distance      176 non-null    float64
 6   trip_duration      176 non-null    float64
 7   avg_trip_distance  176 non-null    float64
 8   avg_trip_duration  176 non-null    float64
 9   fare_amount        132 non-null    float64
 10  total_amount       132 non-null    float64
 11  shared_requests    88 non-null     float64
dtypes: float64(8), int64(3), object(1)
memory usage: 16.6+ KB


In [288]:
df2 = df2[df2['date'] >= '2021-01-01']

In [289]:
df1['date'] = pd.to_datetime(df1['year'].astype(str) + '-' + df1['month'].astype(str), format='%Y-%m')

# Convertir ambas columnas 'date' a datetime
df1['date'] = pd.to_datetime(df1['date'], errors='coerce')
df2['date'] = pd.to_datetime(df2['date'], errors='coerce')

In [ ]:
# Realizar la unión
merged_df = pd.merge(df1, df2, on=['date', 'industry'], how='outer')

# Eliminar columnas duplicadas
df = df.loc[:, ~df.columns.duplicated()]

# Guardamos el DataFrame combinado en un archivo CSV
merged_df.to_csv(r'..\..\datasets\2. Depurados\TLC Aggregated Data\merged_taxi_data.csv', index=False)